In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [2]:
featuresDf = pd.read_pickle('./dataset/featuresDF_1_LN.pkl')
featuresDf['fourier_mfcc'] = [np.concatenate([featuresDf.fourier[i],
                              featuresDf.mfcc[i]]) for i in range(len(featuresDf))]

In [3]:
X = np.array(featuresDf['fourier_mfcc'].tolist())
y = np.array(featuresDf['class'].tolist())

In [6]:
import h2o
from h2o.automl import H2OAutoML
h2o.init(nthreads = -1, max_mem_size = 6)
DF = pd.DataFrame(X, columns=[f'fm{e}' for e in range(1,385)])
DF['class'] = y
hf = h2o.H2OFrame(DF)
y_columns = 'class'

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,19 secs
H2O cluster timezone:,Europe/Paris
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.11
H2O cluster version age:,10 days
H2O cluster name:,H2O_from_python_EcologÃ­a_02pbsy
H2O cluster total nodes:,1
H2O cluster free memory:,6 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [ ]:
# Fitting models (AutoML):
aml_ti = H2OAutoML(max_models= 20, seed= 1, nfolds=5, sort_metric='RMSE')
aml_ti.train(y = y_columns, training_frame = hf) # x = x_columns
lb_ti = aml_ti.leaderboard
print(aml_ti.leader)
model_path = h2o.save_model(model=aml_ti, path="./models", force=True)
h2o.cluster().shutdown()

AutoML progress: |███████████████████████